In [3]:
import re
import pandas as pd
import os

COMM = '~'
START = 'C'
AD1 = "#"
AD2 = "-"
END = 'A'

MTDE_RAW = {'DEPT': 'FT', 'CNOIS': 'mV', 'IS': 'mV', 'COFS': 'mV', 'SR00': 'mV', 'SR01': 'mV', 'SR02': 'mV', 'SR03': 'mV',
      'SR04': 'mV', 'SR05': 'mV', 'SR06': 'mV', 'SR07': 'mV', 'SR08': 'mV', 'SR09': 'mV', 'SR10': 'mV', 'SR11': 'mV',
      'ANOIS': 'mV', 'IL': 'mV', 'AOFS': 'mV', 'LR00': 'mV', 'LR01': 'mV', 'LR02': 'mV', 'LR03': 'mV', 'LR04': 'mV',
      'LR05': 'mV', 'LR06': 'mV', 'LR07': 'mV', 'LR08': 'mV', 'LR09': 'mV', 'LR10': 'mV', 'LR11': 'mV', 'LR12': 'mV',
      'LR13': 'mV', 'LR14': 'mV', 'LR15': 'mV', 'LR16': 'mV', 'LR17': 'mV', 'LR18': 'mV', 'LR19': 'mV', 'LR20': 'mV',
      'LR21': 'mV', 'LR22': 'mV', 'LR23': 'mV', 'LR24': 'mV', 'LR25': 'mV', 'LR26': 'mV', 'LR27': 'mV', 'LR28': 'mV',
      'LR29': 'mV', 'LR30': 'mV', 'LR31': 'mV', 'LR32': 'mV', 'LR33': 'mV', 'LR34': 'mV', 'LR35': 'mV', 'LR36': 'mV',
      'LR37': 'mV', 'LR38': 'mV', 'LR39': 'mV', 'LR40': 'mV', 'LR41': 'mV', 'LR42': 'mV', 'LR43': 'mV', 'LR44': 'mV',
      'LR45': 'mV', 'LR46': 'mV', 'LR47': 'mV', 'LR48': 'mV', 'LR49': 'mV', 'LR50': 'mV', 'LR51': 'mV', 'LR52': 'mV',
      'LR53': 'mV', 'LR54': 'mV', 'LR55': 'mV', 'LR56': 'mV', 'LR57': 'mV', 'LR58': 'mV', 'LR59': 'mV', 'TEMP': 'degF',
      'GR': 'gAPI', 'LSPD': 'FT/M', 'TENS': 'lb'}

MTDE_PROC_4_7_9 = {"DEPT" : "FT", "GR" : "gAPI", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches",
             "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown",
             "SN04" : "Unknown","SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

MTDE_PROC_4_7_9_13 = {"DEPT" : "FT", "GR" : "gAPI", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "THK13" : "inches", "ML13" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches",
             "NTH13" : "inches", "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown",
             "SN04" : "Unknown","SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

MTDE_PROC_2_4_4tbg_7_9 = {"DEPT" : "FT", "GR" : "gAPI", "THK2" : "inches","ML2" : "%", "THK4-TBG" : "inches","ML4-TBG" : "%", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches", "NTH4-TBG" :  "inches","NTH2" :  "inches",
             "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown","SN04" : "Unknown",
             "SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown",
                          "LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

MTDE_PROC_4_4lnr_7_9 = {"DEPT" : "FT", "GR" : "gAPI", "THK4-LNR" : "inches","ML4-LNR" : "%", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches", "NTH4-LNR" :  "inches",
             "THK13" : "inches", "ML13" : "%", "NTH13" : "inches",
             "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown","SN04" : "Unknown",
             "SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

# gathers the headers
ff = []

CURVE_START = False
AFTER_DESC = False
FIN = False

# Output file = op
#  = open("rp_final.las", "w")
debug = 0
filename = 'MP RAW.las'
# Inputting data from las file and going through it line by line
with open(filename, 'r') as td:
    
    #going through line by line
    for line in td:
        if (line[0] == COMM) and (line[1] == END):
            break
        elif (line[0] == COMM) and (line[1] == START):
            CURVE_START = True
#             if (debug):
#                 print(line)


        if CURVE_START:
            if (line[0] == AD1) and (line[1] == AD2):
                AFTER_DESC = True
                continue
                
            if AFTER_DESC:
                currline = line.strip()
                ff.append(currline.split(":"))
        
    
# print(ff)
curves = {}
for x in ff:
    curr = x[0].strip()
    name, unit = curr.split(".")
#     tbQC.append((name, unit))
    curves[name] = unit
print(curves)

{'DEPT': 'FT', 'CNOIS': 'mV', 'IS': 'mV', 'COFS': 'mV', 'SR00': 'mV', 'SR01': 'mV', 'SR02': 'mV', 'SR03': 'mF', 'SR04': 'mV', 'SR05': 'mV', 'SR06': 'mV', 'SR07': 'mV', 'SR08': 'mV', 'SR09': 'mV', 'SR10': 'mV', 'SR11': 'mV', 'ANOIS': 'mV', 'IL': 'mV', 'AOFS': 'mV', 'LR00': 'mV', 'LR01': 'mV', 'LR02': 'mV', 'LR03': 'mV', 'LR04': 'mV', 'LR05': 'mV', 'LR06': 'mV', 'LR07': 'mV', 'LR08': 'mV', 'LR09': 'mV', 'LR10': 'mV', 'LR11': 'mV', 'LR12': 'mV', 'LR13': 'mV', 'LR14': 'mV', 'LR15': 'mV', 'LR16': 'mV', 'LR17': 'mV', 'LR18': 'mV', 'LR19': 'mV', 'LR20': 'mV', 'LR21': 'mV', 'LR22': 'mV', 'LR23': 'mV', 'LR24': 'mV', 'LR25': 'mV', 'LR26': 'mV', 'LR27': 'mV', 'LR28': 'mV', 'LR29': 'mV', 'LR30': 'mV', 'LR31': 'mV', 'LR32': 'mV', 'LR33': 'mV', 'LR34': 'mV', 'LR35': 'mV', 'LR36': 'mV', 'LR37': 'mV', 'LR38': 'mV', 'LR39': 'mV', 'LR40': 'mV', 'LR41': 'mV', 'LR42': 'mV', 'LR43': 'mV', 'LR44': 'mV', 'LR45': 'mV', 'LR46': 'mV', 'LR47': 'mV', 'LR48': 'mV', 'LR49': 'mV', 'LR50': 'mV', 'LR51': 'mV', 'LR52':

In [4]:
checked = {}
dict_ref = MTDE_RAW
totalcurves = 0
for x in curves:
#     print(x, curves[x])
    if (x in dict_ref):
        if(curves[x] == dict_ref[x]):
#             print("values and unit is the same")
#         print(MTDE)
#         print(x, curves[x])
            checked[x] = curves[x]
            totalcurves+=1
        else:
#             pass
            print("issue with the unit for", x , curves[x])
    else:
#         pass
        checked[x] = curves[x]
        print("CURVE ", x, " IS NOT IN ",  filename)

# print(MTDE)
# print(checked)
if dict_ref == checked:
    print("All curvenames and units are correct")
    print(totalcurves, " Curves are in the las file ", filename)
else:
    if(len(dict_ref) < len(checked)):
        print("there might be extra curves")
    elif len(dict_ref) > len(checked):
        print("there are some missing curves")
    
    print("\n\nMISSING CURVES::\n")
    for x in dict_ref:
        if x not in checked:
            print(x, dict_ref[x])
    print("\n\nEXTRA CURVES::\n")
    for m in checked:
            if m not in dict_ref:
                print(m, checked[x])

issue with the unit for SR03 mF
there are some missing curves


MISSING CURVES::

SR03 mV


EXTRA CURVES::

